# Importação dos dados

In [223]:
"""
Geral
1. Quantos restaurantes únicos estão registrados?
2. Quantos países únicos estão registrados?
3. Quantas cidades únicas estão registradas?
4. Qual o total de avaliações feitas?
5. Qual o total de tipos de culinária registrados?
"""

'\nGeral\n1. Quantos restaurantes únicos estão registrados?\n2. Quantos países únicos estão registrados?\n3. Quantas cidades únicas estão registradas?\n4. Qual o total de avaliações feitas?\n5. Qual o total de tipos de culinária registrados?\n'

Importação de bibliotecas

In [224]:
import pandas as pd
import sys
import numpy
import inflection
import plotly.express as px


In [225]:
# Funções:


def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
    
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]



def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df



# Extração dos dados e limpeza dos dados

### Extração

In [226]:
df_raw = pd.read_csv("zomato.csv")

In [227]:
df = df_raw.copy()

### Limpeza dos dados

In [228]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [229]:
df = rename_columns(df)

In [230]:
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}



In [231]:
# Mudança de coluna
df['country_code'] = df['country_code'].map(COUNTRIES)

# Criação de uma nova coluna cost range
df["cost_range"] = df.loc[:, "price_range"].apply(lambda x : create_price_tye(x))

In [232]:
df["cuisines"] = df["cuisines"].astype(str)
df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

In [233]:
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'cost_range'],
      dtype='object')

In [234]:
df.head()
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'cost_range'],
      dtype='object')

In [235]:


CURRENCIES = {
"Botswana Pula(P)": 0.074,
"Brazilian Real(R$)": 0.20,
"Dollar($)": 1,
"Emirati Diram(AED)": 0.27,
"Indian Rupees(Rs.)": 0.012,
"Indonesian Rupiah(IDR)": 0.000063,
"NewZealand($)": 0.61,
"Pounds(£)": 1.27,
"Qatari Rial(QR)": 0.27,
"Rand(R)": 0.054,
"Sri Lankan Rupee(LKR)": 0.0032,
"Turkish Lira(TL)": 0.033
}


df['exchange_to_dolar'] = df['currency'].map(CURRENCIES)

df['average_cost_for_two_dolar'] = df['average_cost_for_two'] * df['exchange_to_dolar']




In [236]:
df = df.loc[(df["average_cost_for_two_dolar"] < 1000000), :].copy()


In [237]:
df.head()

df = df.astype({"restaurant_id": int})

# Geral

## 1. Quantos restaurantes únicos estão registrados?


In [238]:
df.loc[:, "restaurant_id"].nunique()

6941

## 2. Quantos países únicos estão registrados?

In [239]:
df.loc[:, "country_code"].nunique()


15

## 3. Quantas cidades únicas estão registradas?

In [240]:
df.loc[:, "city"].nunique()


125

## 4.Qual o total de avaliações feitas?

In [241]:
len(df.loc[:, "aggregate_rating"])

7526

## 5. Qual o total de tipos de culinária registrados?

In [242]:
# Não sei se tá certo...
len(df["cuisines"].unique())

166

# Pais

## 1. Qual o nome do país que possui mais cidades registradas?

In [243]:
df_aux = df.loc[:, ["country_code", "city"]].groupby(["country_code"]).nunique().reset_index()
pais_mais_cidades = df_aux.max()
print(pais_mais_cidades)

country_code    United States of America
city                                  49
dtype: object


## 2. Qual o nome do país que possui mais restaurantes registrados?

In [244]:
df_aux = df.loc[:, ["country_code", "restaurant_id"]].groupby(["country_code"]).nunique().reset_index()
pais_mais_restaurantes = df_aux.max()
print(pais_mais_restaurantes)

country_code     United States of America
restaurant_id                        3120
dtype: object


## 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [245]:

# Voltar daqui (21/01/2024)
df_aux = df.loc[ df["price_range"] == 4, ["restaurant_id", "country_code"]].groupby("country_code"). count()
df_aux.reset_index()
df_aux         

,restaurant_id
country_code,
Australia,33
Brazil,159
Canada,54
England,83
India,210
Indonesia,18
New Zeland,81
Philippines,23
Qatar,63


## 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [246]:
df_aux = df.loc[:, ["cuisines", "country_code"]].groupby("country_code").nunique()
df_aux.reset_index()

,country_code,cuisines
0,Australia,43
1,Brazil,43
2,Canada,42
3,England,52
4,India,78
5,Indonesia,21
6,New Zeland,37
7,Philippines,14
8,Qatar,31
9,Singapure,21


## 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [247]:
# aggregate_rating
df.loc[:, ["country_code", "aggregate_rating"]].groupby("country_code").count().reset_index()

,country_code,aggregate_rating
0,Australia,179
1,Brazil,261
2,Canada,180
3,England,437
4,India,3507
5,Indonesia,82
6,New Zeland,253
7,Philippines,88
8,Qatar,94
9,Singapure,82


## 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [248]:
df_aux = df.loc[:, ["country_code", "has_online_delivery"]].groupby(["country_code"]).count()
df_aux

,has_online_delivery
country_code,
Australia,179
Brazil,261
Canada,180
England,437
India,3507
Indonesia,82
New Zeland,253
Philippines,88
Qatar,94


## 7.Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [249]:
df_aux = df.loc[:, ["country_code", "has_table_booking"]].groupby("country_code").count().reset_index()
df_aux

,country_code,has_table_booking
0,Australia,179
1,Brazil,261
2,Canada,180
3,England,437
4,India,3507
5,Indonesia,82
6,New Zeland,253
7,Philippines,88
8,Qatar,94
9,Singapure,82


## 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [250]:
df.loc[:, ["votes", "country_code"]].groupby("country_code").mean().reset_index().max()

country_code    United States of America
votes                        1115.317073
dtype: object

## 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [251]:
#aggregate_rating
df.loc[:, ["aggregate_rating", "country_code"]].groupby("country_code").mean().reset_index().max()

country_code        United States of America
aggregate_rating                         4.6
dtype: object

## 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [252]:
df.loc[:, ["aggregate_rating", "country_code"]].groupby("country_code").mean().reset_index().min()


country_code        Australia
aggregate_rating     3.349425
dtype: object

## 11. Qual a média de preço de um prato para dois por país?

In [253]:
df.loc[:, ["country_code", "average_cost_for_two_dolar"]].groupby(["country_code"]).mean().reset_index()

,country_code,average_cost_for_two_dolar
0,Australia,71.195531
1,Brazil,27.475096
2,Canada,41.861111
3,England,54.668124
4,India,8.427648
5,Indonesia,19.699024
6,New Zeland,37.480040
7,Philippines,89.620727
8,Qatar,47.695213
9,Singapure,141.036585


# Cidade

## 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [254]:
df.loc[:, ["city", "restaurant_id"]].groupby(["city"]).nunique().reset_index().max()


city             İstanbul
restaurant_id          80
dtype: object

## 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [255]:
linhas_selecionadas = df["aggregate_rating"] > 4
df.loc[linhas_selecionadas, ["city", "aggregate_rating"]].groupby(["city"]).mean().reset_index().max()


city                İstanbul
aggregate_rating         4.9
dtype: object

## 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [256]:
linhas_selecionadas = df["aggregate_rating"] <2.5
df.loc[linhas_selecionadas, ["city", "aggregate_rating"]].groupby(["city"]).mean().reset_index()

,city,aggregate_rating
0,Ankara,2.100000
1,Brasília,0.000000
2,Clarens,0.000000
3,Colombo,2.300000
4,Durban,2.200000
5,Edinburgh,0.000000
6,Gangtok,0.000000
7,Hamilton,2.350000
8,Manchester,0.000000
9,Nashik,2.300000


## 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [257]:
df.loc[:, ["price_range", "city"]].groupby(["city"]).mean().reset_index().min()

city           Abu Dhabi
price_range          1.5
dtype: object

## 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [258]:
df.loc[:, ["city", "cuisines"]].groupby("city").nunique().reset_index().max()

city        İstanbul
cuisines          32
dtype: object

## 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [259]:
df.loc[:, ["has_table_booking", "city"]].groupby(["city"]).count().reset_index().max()

city                 İstanbul
has_table_booking          99
dtype: object

## 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [260]:
df.loc[:, ["is_delivering_now", "city"]].groupby(["city"]).count().reset_index().max()



city                 İstanbul
is_delivering_now          99
dtype: object

## 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [261]:
df.loc[:, ["has_online_delivery", "city"]].groupby(["city"]).count().reset_index().max()


city                   İstanbul
has_online_delivery          99
dtype: object

# Restaurantes

## 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [262]:
df.loc[:, ["restaurant_id", "votes"]].groupby(["restaurant_id"]).sum().reset_index().max()

restaurant_id    19040277
votes               41333
dtype: int64

## 2. Qual o nome do restaurante com a maior nota média?

In [263]:
df.loc[:, ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index().max()


restaurant_name     ह-tea The Tea Hut
aggregate_rating                  4.9
dtype: object

## 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [264]:
df_aux = df.loc[:, ["restaurant_name", "average_cost_for_two"]].groupby(["restaurant_name"]).max().reset_index()
df_aux.max()

restaurant_name         ह-tea The Tea Hut
average_cost_for_two              1200000
dtype: object

## 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [265]:
df_aux = df.loc[df["cuisines"] == "Brazilian", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.loc[df_aux["aggregate_rating"].idxmin()]

restaurant_name     Bar do Luiz Fernandes
aggregate_rating                      0.0
Name: 6, dtype: object

## 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [266]:
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'cost_range', 'exchange_to_dolar',
       'average_cost_for_two_dolar'],
      dtype='object')

In [267]:
df_aux = df.loc[(df["cuisines"] == "Brazilian") & (df["country_code"] == "Brazil"), ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.loc[df_aux["aggregate_rating"].idxmax()]

restaurant_name     Aprazível
aggregate_rating          4.9
Name: 2, dtype: object

## 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [268]:
df.loc[:, ["has_online_delivery", "votes"]].groupby(["has_online_delivery"]).mean().reset_index()

,has_online_delivery,votes
0,0,493.104658
1,1,843.027516


## 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [269]:
df.loc[:, ["has_table_booking", "average_cost_for_two"]].groupby(["has_table_booking"]).mean().reset_index()

,has_table_booking,average_cost_for_two
0,0,3454.031591
1,1,9533.042827


In [270]:
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'cost_range', 'exchange_to_dolar',
       'average_cost_for_two_dolar'],
      dtype='object')

## 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [271]:
# numpy.set_printoptions(threshold=sys.maxsize)
# for item in df["cuisines"].unique():
#     if "BBQ" in item:
#         print(item)

In [272]:
linhas_selecionadas = ((df["country_code"] == "United States of America") & ((df["cuisines"] == "BBQ") | (df["cuisines"] == "Japanese")))
df_aux = (df.loc[linhas_selecionadas, ["cuisines", "average_cost_for_two"]]
          .groupby(["cuisines"])
          .mean()
          .reset_index())

df_aux

# Valores muito próximos


,cuisines,average_cost_for_two
0,BBQ,39.642857
1,Japanese,56.406250


# Culinária

## 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [273]:
df_aux = df.loc[df["cuisines"] == "Italian",["restaurant_name","aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.loc[df["aggregate_rating"].idxmax()]


restaurant_name     Avenida Paulista
aggregate_rating                 0.0
Name: 17, dtype: object

## 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [274]:
df_aux = df.loc[df["cuisines"] == "Italian",["restaurant_name","aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.loc[df["aggregate_rating"].idxmin()]

restaurant_name     Frenchie's Italian Cuisine
aggregate_rating                           4.6
Name: 105, dtype: object

## 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [275]:
df_aux = df.loc[df["cuisines"] == "American",["restaurant_name","aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.max()
#df_aux.loc[df["aggregate_rating"].idxmax()]

restaurant_name     Zingerman's Roadhouse
aggregate_rating                      4.9
dtype: object

## 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [276]:
df_aux = df.loc[df["cuisines"] == "American",["restaurant_name","aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.min()


restaurant_name     11th Street Diner
aggregate_rating                  0.0
dtype: object

In [277]:
df.loc[df["cuisines"] == "Brazilian", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).min().reset_index().min()

restaurant_name     Academia da Cachaça
aggregate_rating                    0.0
dtype: object

In [278]:
# Voltar aqui depois pra trocar o country_code
df.loc[(df["cuisines"] == "Brazilian") & (df["country_code"] == "Brazil"), ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).max().reset_index().max()


restaurant_name     Zazá Bistrô Tropical
aggregate_rating                     4.9
dtype: object

## 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [279]:
df_aux = df.loc[df["cuisines"] == "Arabian", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
indice_min = df_aux["aggregate_rating"].idxmin()
df_aux.loc[indice_min]

restaurant_name     Raful
aggregate_rating      0.0
Name: 26, dtype: object

## 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [280]:
# cuisines = Arabian

# PRECISO REVER OS CASOS QUE O ARABIAN NÃO TÁ SOZINHO NA LISTA

df_aux = df.loc[df["cuisines"] == "Arabian", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()

indice_min = df_aux["aggregate_rating"].idxmin()
df_aux.loc[indice_min]


restaurant_name     Raful
aggregate_rating      0.0
Name: 26, dtype: object

## 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [281]:
df_aux = df.loc[df["cuisines"] == "Japanese", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()

indice_min = df_aux["aggregate_rating"].idxmax()
df_aux.loc[indice_min]

restaurant_name     Chotto Matte
aggregate_rating             4.9
Name: 9, dtype: object

## 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [282]:
df_aux = df.loc[df["cuisines"] == "Japanese", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.loc[df_aux["aggregate_rating"].idxmin()]

restaurant_name     Banzai Sushi
aggregate_rating             0.0
Name: 5, dtype: object

## 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [283]:
# Home-made
df_aux = df.loc[df["cuisines"] == "Home-made", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux.loc[df_aux["aggregate_rating"].idxmin()]

restaurant_name     GurMekan Restaurant
aggregate_rating                    3.7
Name: 0, dtype: object

## 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [284]:
df_aux = df.loc[df["cuisines"] == "Home-made", ["restaurant_name", "aggregate_rating"]].groupby(["restaurant_name"]).mean().reset_index()
df_aux
#df_aux.loc[df_aux["aggregate_rating"].idxmax()]

,restaurant_name,aggregate_rating
0,GurMekan Restaurant,3.7
1,Kanaat Lokantası,4.0


## 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [285]:
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'cost_range', 'exchange_to_dolar',
       'average_cost_for_two_dolar'],
      dtype='object')

In [286]:
df.loc[:,["cuisines", "average_cost_for_two"]].groupby(["cuisines"]).mean().reset_index().max()

cuisines                          nan
average_cost_for_two    333333.333333
dtype: object

## 12. Qual o tipo de culinária que possui a maior nota média?

In [287]:
df.loc[:,["cuisines", "aggregate_rating"]].groupby(["cuisines"]).mean().reset_index().max()


cuisines            nan
aggregate_rating    4.9
dtype: object

## 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [288]:
linhas_selecionadas = ((df["has_online_delivery"] == 1) & (df["is_delivering_now"])) == 1
df.loc[linhas_selecionadas, ["cuisines", "restaurant_id"]].groupby(["cuisines"]).nunique().reset_index().max()

cuisines         Tibetan
restaurant_id        317
dtype: object

In [289]:
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,cost_range,exchange_to_dolar,average_cost_for_two_dolar
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,expensive,0.074,81.4
1,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,expensive,0.074,81.4
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,gourmet,0.074,229.4
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,3,4.4,5BA829,Very Good,867,expensive,0.074,59.2
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,3,4.4,5BA829,Very Good,858,expensive,0.074,51.8


In [290]:
print(df["country_code"].unique())

['Philippines' 'Brazil' 'Australia' 'United States of America' 'Canada'
 'Singapure' 'United Arab Emirates' 'India' 'Indonesia' 'New Zeland'
 'England' 'Qatar' 'South Africa' 'Sri Lanka' 'Turkey']


# No freestyle

### Top mais cidade com média pedidos mais caros

In [291]:
df_aux = df.loc[:, ["price_range", "country_code", "city"]].groupby(["city","country_code"]).mean().reset_index()
df_aux = df_aux.sort_values(by="price_range", ascending=False).reset_index(drop=True)
df_aux.head(10)

,city,country_code,price_range
0,Midrand,South Africa,4.000000
1,Pasay City,Philippines,4.000000
2,East Rand,South Africa,4.000000
3,Roodepoort,South Africa,4.000000
4,Johannesburg South,South Africa,4.000000
5,Johannesburg,South Africa,3.785714
6,Sandton,South Africa,3.775000
7,Clarens,South Africa,3.730769
8,Singapore,Singapure,3.658537
9,Doha,Qatar,3.627660


In [292]:
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,cost_range,exchange_to_dolar,average_cost_for_two_dolar
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,expensive,0.074,81.4
1,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,expensive,0.074,81.4
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,gourmet,0.074,229.4
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,3,4.4,5BA829,Very Good,867,expensive,0.074,59.2
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,3,4.4,5BA829,Very Good,858,expensive,0.074,51.8


### Quantidade de Restaurantes Registrados por País

In [293]:
df_aux = df.loc[:, ["country_code", "restaurant_id"]].groupby(["country_code"]).nunique().reset_index()
df_aux = df_aux.sort_values(by = "restaurant_id", ascending=False) .reset_index(drop=True)
fig = px.bar(df_aux, 
                    x="country_code", 
                    y ="restaurant_id", 
                    color="country_code", 
                    labels={"aggregate_rating": "Nota avaliação média", "city" : "Cidades", "country_code": "País"})
fig.show()

### Quantidade de Cidades Registradas por País


In [294]:
df_aux = df.loc[:, ["country_code", "city"]].groupby(["country_code"]).nunique().reset_index()
df_aux = df_aux.sort_values(by = "city", ascending=False) .reset_index(drop=True)
fig = px.bar(df_aux, 
                    x="country_code", 
                    y ="city", 
                    color="country_code", 
                    labels={"city": "Quantidade de Cidades", "country_code": "País"})
fig.show()

### Média de Avaliações Feita por País


In [295]:
df_aux = df.loc[:, ["aggregate_rating", "country_code"]].groupby("country_code").mean().reset_index()
df_aux.sort_values(by="aggregate_rating", ascending=False)

,country_code,aggregate_rating
5,Indonesia,4.600000
7,Philippines,4.462500
9,Singapure,4.429268
14,United States of America,4.404659
0,Australia,4.370950
2,Canada,4.321667
12,Turkey,4.313939
8,Qatar,4.267021
6,New Zeland,4.166798
11,Sri Lanka,4.070115


### Média de Preço de um Prato para 2 Pessoas por País

In [296]:
df_aux = df.loc[:, ["country_code", "average_cost_for_two"]].groupby(["country_code"]).mean().reset_index()
df_aux = df_aux.sort_values(by="average_cost_for_two").reset_index(drop=True)
df_aux

# fig = px.bar(df_aux, 
#                     x="country_code", 
#                     y ="city", 
#                     color="country_code", 
#                     labels={"city": "Quantidade de Cidades", "country_code": "País"})
# fig.show()


,country_code,average_cost_for_two
0,Canada,41.861111
1,England,43.045767
2,United States of America,54.921864
3,New Zeland,61.442688
4,Australia,71.195531
5,Turkey,128.848485
6,Brazil,137.375479
7,Singapure,141.036585
8,United Arab Emirates,158.218563
9,Qatar,176.648936


In [297]:
# Transformar em dolar
df.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'cost_range', 'exchange_to_dolar',
       'average_cost_for_two_dolar'],
      dtype='object')

In [298]:
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,cost_range,exchange_to_dolar,average_cost_for_two_dolar
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,expensive,0.074,81.4
1,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,expensive,0.074,81.4
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,gourmet,0.074,229.4
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,3,4.4,5BA829,Very Good,867,expensive,0.074,59.2
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,3,4.4,5BA829,Very Good,858,expensive,0.074,51.8


### Detectando um outlier

In [299]:
df_aus = df.loc[df["country_code"] == "Australia", ["currency", "average_cost_for_two"]]
df_aus.sort_values(by="average_cost_for_two", ascending=False)

,currency,average_cost_for_two
630,Dollar($),250
639,Dollar($),200
655,Dollar($),170
650,Dollar($),165
631,Dollar($),160
...,...,...
637,Dollar($),30
646,Dollar($),20
1603,Dollar($),0
1569,Dollar($),0


In [302]:
df_aux = df_aux.loc[:, ["country_code", "average_cost_for_two_dolar"]].groupby(["country_code"]).mean().reset_index()
df_aux = df_aux.sort_values(by="average_cost_for_two_dolar", ascending=False)
df_aux


,country_code,average_cost_for_two_dolar
9,Singapure,141.036585
7,Philippines,89.620727
0,Australia,71.195531
14,United States of America,54.921864
3,England,54.668124
8,Qatar,47.695213
13,United Arab Emirates,42.719012
2,Canada,41.861111
6,New Zeland,37.480040
1,Brazil,27.475096
